In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# 시각화 관련
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from collections import Counter
from PIL import Image
from matplotlib import rc

# 그래프에서 한글 폰트 깨지는 문제에 대한 대처(전역 글꼴 설정)
import matplotlib.font_manager as fm
import warnings
warnings.filterwarnings(action='ignore') 

import matplotlib
matplotlib.rcParams['font.family'] ='Malgun Gothic'
matplotlib.rcParams['axes.unicode_minus'] =False

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier, NeighborhoodComponentsAnalysis
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

#import EDA

from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

#nlp
# from bs4 import BeautifulSoup
# from urllib.request import urlopen
# import requests
# from selenium import webdriver
# from tqdm.notebook import tqdm
# import time, urllib
# from selenium import webdriver
# from selenium.webdriver.common.keys import Keys
# keys = Keys()
# import re, os, sys, json

# from hanspell import spell_checker

# from konlpy.tag import Kkma #형태소분석
# from konlpy.tag import Okt #형태소분석
# import soynlp; from soynlp.normalizer import * #정규화
# # from pykospacing import Spacing------------------>써야함!
# # spacing = Spacing()------------------------------>써야함!

# from konlpy.tag import Komoran, Hannanum, Kkma, Okt
# komoran = Komoran(); hannanum = Hannanum(); kkma = Kkma(); okt = Okt();

In [6]:
df = pd.read_pickle('data/real_final_df_230115.pickle')
df =df.astype({'shoulder_big':'int','shoulder_small':'int',
               'chongjang_big':'int','chongjang_small':'int',
               'total_big':'int','total_small':'int',
               'arm_big':'int','arm_small':'int',
               'chest_big':'int','chest_small':'int',})
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19853 entries, 0 to 20036
Data columns (total 33 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   user               19853 non-null  object 
 1   gender             19853 non-null  object 
 2   height             19853 non-null  float64
 3   weight             19853 non-null  float64
 4   item               19853 non-null  object 
 5   size               19853 non-null  object 
 6   star               19853 non-null  float64
 7   content            19853 non-null  object 
 8   size_eval          19853 non-null  object 
 9   bright_eval        19853 non-null  object 
 10  color_eval         19853 non-null  object 
 11  thick_eval         19853 non-null  object 
 12  cm                 19853 non-null  object 
 13  총장                 19853 non-null  float64
 14  어깨너비               19035 non-null  float64
 15  가슴단면               19853 non-null  float64
 16  소매길이               198

In [11]:
df['total_big'].head()

0    1
1    0
2    0
3    0
4    0
Name: total_big, dtype: int32

In [28]:

    
df_total_big        = df[df['total_big'] == 1]
df_total_small      = df[df['total_small'] == 1]
df_total_soso       = df[(df['total_big'] ==0) & (df['total_small'] ==0)]

df_chongjang_big    = df[df['chongjang_big'] == 1]
df_chongjang_small  = df[df['chongjang_small'] == 1]
df_chongjang_soso   = df[(df['chongjang_big'] == 0) & (df['chongjang_small'] == 0)]

df_shoulder_big     = df[df['shoulder_big'] == 1]
df_shoulder_small   = df[df['shoulder_small'] == 1]
df_shoulder_soso    = df[(df['shoulder_big'] == 0) & (df['shoulder_small'] == 0)]

df_chest_big        = df[df['chest_big'] == 1]
df_chest_small      = df[df['chest_small'] == 1]
df_chest_soso       = df[(df['chest_big'] == 0 )& (df['chest_small'] == 0)]

df_arm_big          = df[df['arm_big'] == 1]
df_arm_small        = df[df['arm_small'] == 1]
df_arm_soso         = df[(df['arm_big'] == 0) & (df['arm_small'] == 0)]

###
df_total_big        = df_total_big[['height','weight','총장','어깨너비','가슴단면','소매길이']]
df_total_small      = df_total_small[['height','weight','총장','어깨너비','가슴단면','소매길이']]
df_total_soso       = df_total_soso[['height','weight','총장','어깨너비','가슴단면','소매길이']]

df_chongjang_big    = df_chongjang_big[['height','weight','총장']]
df_chongjang_small  = df_chongjang_small[['height','weight','총장']]
df_chongjang_soso   = df_chongjang_soso[['height','weight','총장']]

df_shoulder_big     = df_shoulder_big[['height','weight','어깨너비']]
df_shoulder_small   = df_shoulder_small[['height','weight','어깨너비']]
df_shoulder_soso    = df_shoulder_soso[['height','weight','어깨너비']]

df_chest_big        = df_chest_big[['height','weight','가슴단면']]
df_chest_small      = df_chest_small[['height','weight','가슴단면']]
df_chest_soso       = df_chest_soso[['height','weight','가슴단면']]

df_arm_big          = df_arm_big[['height','weight','소매길이']]
df_arm_small        = df_arm_small[['height','weight','소매길이']]
df_arm_soso         = df_arm_soso[['height','weight','소매길이']]


print('---크다. 작다. 보통---')
print('전체{},{},{}'.format(df_total_big.shape,df_total_small.shape,df_total_soso.shape))
print('총장{},{},{}'.format(df_chongjang_big.shape,df_chongjang_small.shape,df_chongjang_soso.shape))
print('어깨{},{},{}'.format(df_shoulder_big.shape,df_shoulder_small.shape,df_shoulder_soso.shape))
print('가슴{},{},{}'.format(df_chest_big.shape,df_chest_small.shape,df_chest_soso.shape))
print('팔{},{},{}'.format(df_arm_big.shape,df_arm_small.shape,df_arm_soso.shape))

df_total_big.dropna(axis = 0,inplace = True)
df_total_small.dropna(axis = 0,inplace = True)
df_total_soso.dropna(axis = 0,inplace = True)

df_chongjang_big.dropna(axis = 0,inplace = True)
df_chongjang_small.dropna(axis = 0,inplace = True)
df_chongjang_soso.dropna(axis = 0,inplace = True)

df_arm_big.dropna(axis = 0,inplace = True)
df_arm_small.dropna(axis = 0,inplace = True)
df_arm_soso.dropna(axis = 0,inplace = True)

df_chest_big.dropna(axis = 0,inplace = True)
df_chest_small.dropna(axis = 0,inplace = True)
df_chest_soso.dropna(axis = 0,inplace = True)

df_shoulder_big.dropna(axis = 0,inplace = True)
df_shoulder_small.dropna(axis = 0,inplace = True)
df_shoulder_soso.dropna(axis = 0,inplace = True)
print('---크다. 작다. 보통---')
print('전체{},{},{}'.format(df_total_big.shape,df_total_small.shape,df_total_soso.shape))
print('총장{},{},{}'.format(df_chongjang_big.shape,df_chongjang_small.shape,df_chongjang_soso.shape))
print('어깨{},{},{}'.format(df_shoulder_big.shape,df_shoulder_small.shape,df_shoulder_soso.shape))
print('가슴{},{},{}'.format(df_chest_big.shape,df_chest_small.shape,df_chest_soso.shape))
print('팔{},{},{}'.format(df_arm_big.shape,df_arm_small.shape,df_arm_soso.shape))


#리스트에 들어가는 순서는 big,small,soso 순!! 꼭지키기!!
hood_chongjang_train_lst = [
                 #df_total_big,df_total_small,df_total_soso,
                  df_chongjang_big,df_chongjang_small,df_chongjang_soso,
                  #df_shoulder_big,df_shoulder_small,df_shoulder_soso,
                  #df_chest_big,df_chest_small,df_chest_soso,
                  #df_arm_big,df_arm_small,df_arm_soso
                  ]
hood_shoulder_train_lst = [
                 #df_total_big,df_total_small,df_total_soso,
                  #df_chongjang_big,df_chongjang_small,df_chongjang_soso,
                  df_shoulder_big,df_shoulder_small,df_shoulder_soso,
                  #df_chest_big,df_chest_small,df_chest_soso,
                  #df_arm_big,df_arm_small,df_arm_soso
                  ]

hood_chest_train_lst = [
                 #df_total_big,df_total_small,df_total_soso,
                  #df_chongjang_big,df_chongjang_small,df_chongjang_soso,
                  #df_shoulder_big,df_shoulder_small,df_shoulder_soso,
                  df_chest_big,df_chest_small,df_chest_soso,
                  #df_arm_big,df_arm_small,df_arm_soso
                  ]

hood_arm_train_lst = [
                 #df_total_big,df_total_small,df_total_soso,
                  #df_chongjang_big,df_chongjang_small,df_chongjang_soso,
                  #df_shoulder_big,df_shoulder_small,df_shoulder_soso,
                  #df_chest_big,df_chest_small,df_chest_soso,
                  df_arm_big,df_arm_small,df_arm_soso
                  ]
#전체적으로 크다 혹은 작다 라고 답한 사람들은 그냥 총장, 어깨, 가슴, 팔에 전부 1을 주는 것이 어떤지,,
#아니면 전체모델을 따로 돌리던가,, 하 모르겄다

---크다. 작다. 보통---
전체(3942, 6),(248, 6),(15663, 6)
총장(657, 3),(331, 3),(18865, 3)
어깨(200, 3),(34, 3),(19619, 3)
가슴(115, 3),(25, 3),(19713, 3)
팔(458, 3),(28, 3),(19367, 3)
---크다. 작다. 보통---
전체(3798, 6),(237, 6),(15000, 6)
총장(657, 3),(331, 3),(18865, 3)
어깨(191, 3),(32, 3),(18812, 3)
가슴(115, 3),(25, 3),(19713, 3)
팔(458, 3),(28, 3),(19366, 3)


## 훈련

In [38]:
import joblib
def trainModel(lst,sizetype):
    user_pred_list = []
    for i in range(len(lst)):
        X = lst[i].iloc[:, :-1]
        y = lst[i].iloc[:, -1]

        xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size = 0.2, random_state = 531)
        
        mseOos = []
        nTreeList = range(50, 500, 10)

        for iTrees in tqdm(nTreeList, desc='iterate list'):
            depth = None
            RFModel = RandomForestRegressor(n_estimators=iTrees,
                                            max_depth=depth,
                                            oob_score=False, 
                                            random_state=531)
            RFModel.fit(xTrain, yTrain)
            
            #데이터 세트에 대한 MSE 누적
            prediction = RFModel.predict(xTest)
            mseOos.append(mean_squared_error(yTest, prediction))
            
        regr = RandomForestRegressor(n_estimators = nTreeList[np.argmin(mseOos)],
                                random_state=531)
        regr.fit(xTrain, yTrain)
        
        prediction = regr.predict(xTest)
        if i % 3 == 1:
            print("MSE of small {}'s Model : {}".format(lst[i].columns[-1], mean_squared_error(yTest, prediction)))
            joblib.dump(regr, './'+sizetype+'_small_RFModel.pkl')
        elif i % 3 == 2:
            print("MSE of soso {}'s Model : {}".format(lst[i].columns[-1], mean_squared_error(yTest, prediction)))
            joblib.dump(regr, './'+sizetype+'_soso_RFModel.pkl')
        elif i % 3 == 0:
            print("MSE of big {}'s Model : {}".format(lst[i].columns[-1], mean_squared_error(yTest, prediction)))
            joblib.dump(regr, './'+sizetype+'_big_RFModel.pkl')
            
        userTest = [[178, 76]]
        prediction_user = regr.predict(userTest)
        user_pred_list.append(prediction_user)
        print("User Test of {}'s Model : {}".format(lst[i].columns[-1], prediction_user))
    

In [39]:
sizetype = 'chongjang'
trainModel(hood_chongjang_train_lst,sizetype)

iterate list: 100%|██████████| 45/45 [00:24<00:00,  1.87it/s]


MSE of big 총장's Model : 18.818416058042246
User Test of 총장's Model : [71.31904762]


iterate list: 100%|██████████| 45/45 [00:19<00:00,  2.26it/s]


MSE of small 총장's Model : 71.30255563849123
User Test of 총장's Model : [69.684]


iterate list: 100%|██████████| 45/45 [02:07<00:00,  2.84s/it]


MSE of soso 총장's Model : 13.186106683936623
User Test of 총장's Model : [71.26602641]


In [ ]:
sizetype = 'chest'
trainModel(hood_chest_train_lst,sizetype)
sizetype = 'arm'
trainModel(hood_arm_train_lst,sizetype)
sizetype = 'shoulder'
trainModel(hood_shoulder_train_lst,sizetype)

## 230116 회의 중 보기 및 모델 trial

In [6]:
print('총장 크다작다 둘다 1:{}'.format(df[(df['chongjang_big']==1) & (df['chongjang_small']==1)].shape))
print('어깨 크다작다 둘다 1:{}'.format(df[(df['shoulder_big']==1) & (df['shoulder_small']==1)].shape))
print('가슴 크다작다 둘다 1:{}'.format(df[(df['chest_big']==1) & (df['chest_small']==1)].shape))
print('소매 크다작다 둘다 1:{}'.format(df[(df['arm_big']==1) & (df['arm_small']==1)].shape))

총장 크다작다 둘다 1:(66, 33)
어깨 크다작다 둘다 1:(16, 33)
가슴 크다작다 둘다 1:(11, 33)
소매 크다작다 둘다 1:(12, 33)


## 230119 model trial2

In [7]:
df = pd.read_pickle('data/Modeling_DF_230116.pickle')
#df.drop(['total_normal','chongjang_normal','arm_normal','shoulder_normal','chest_normal'],axis = 1)
df.head()

,gender,height,weight,size,content,총장,어깨너비,가슴단면,소매길이,total_big,total_small,chongjang_big,chongjang_small,arm_big,arm_small,chest_big,chest_small,shoulder_big,shoulder_small,total_normal,chongjang_normal,arm_normal,shoulder_normal,chest_normal
0,0,160.0,42.0,M,면이 톡톡해서 좋아요. 너무 얇지도 두껍지도 않아요...평상시 S입는데 오버핏으로 ...,70.0,63.0,59.0,55.0,1,0,1,0,1,0,1,0,1,0,0,1,1,1,1
1,1,174.0,78.0,L,자주사용할 회색후드티 찾고있었는데 가성비도 좋고 기본템으로 활용하기 좋아 구매했어요!,72.0,64.0,61.0,56.0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1
2,1,165.0,63.0,M,요즘은 진짜 와릿이즌이 대세인가봐요.직장동료가 요즘애들이 좋아하는 브랜드라고 해서 ...,70.0,63.0,59.0,55.0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1
3,0,166.0,51.0,S,질 맞고 좋아요 색깔도 이쁘고 촌스럽지 않고 너무 좋아요,68.0,62.0,57.0,54.0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1
4,0,162.0,47.0,S,솔직히 받고나서는 건조기 돌릴거 감안해서 m살까 했는데 지금보니 s사길 잘한 것 같...,68.0,62.0,57.0,54.0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1


In [33]:
def preprocessing(df):
    df = df.astype({'chongjang_big':'int',
                'chongjang_small':'int',
                'shoulder_big':'int',
                'shoulder_small':'int',
                'chest_big':'int',
                'chest_small':'int',
                'arm_big':'int',
                'arm_small':'int'})
    df = df.loc[:, ['gender', 'height', 'weight', 'size', 'content',
                    '총장', '어깨너비', '가슴단면', '소매길이',
                    'chongjang_big', 'chongjang_small',
                    'shoulder_big', 'shoulder_small',
                    'chest_big', 'chest_small',
                    'arm_big', 'arm_small']]
    
    # Make train set
    df_chongjang_big    = df.loc[df['chongjang_big'] == 1, ['height','weight','gender', '총장']]
    df_chongjang_small  = df.loc[df['chongjang_small'] == 1, ['height','weight','gender', '총장']]
    df_chongjang_soso   = df.loc[(df['chongjang_big'] == 0) & (df['chongjang_small'] == 0), ['height','weight','gender', '총장']]

    df_shoulder_big     = df.loc[df['shoulder_big'] == 1, ['height','weight','gender', '어깨너비']]
    df_shoulder_small   = df.loc[df['shoulder_small'] == 1, ['height','weight','gender', '어깨너비']]
    df_shoulder_soso    = df.loc[(df['shoulder_big'] == 0) & (df['shoulder_small'] == 0), ['height','weight','gender', '어깨너비']]

    df_chest_big        = df.loc[df['chest_big'] == 1, ['height','weight','gender', '가슴단면']]
    df_chest_small      = df.loc[df['chest_small'] == 1, ['height','weight','gender', '가슴단면']]
    df_chest_soso       = df.loc[(df['chest_big'] == 0 )& (df['chest_small'] == 0), ['height','weight','gender', '가슴단면']]

    df_arm_big          = df.loc[df['arm_big'] == 1, ['height','weight','gender','소매길이']]
    df_arm_small        = df.loc[df['arm_small'] == 1, ['height','weight','gender','소매길이']]
    df_arm_soso         = df.loc[(df['arm_big'] == 0) & (df['arm_small'] == 0), ['height','weight','gender','소매길이']]

    # Drop Null value
    df_chongjang_big.dropna(axis = 0,inplace = True)
    df_chongjang_small.dropna(axis = 0,inplace = True)
    df_chongjang_soso.dropna(axis = 0,inplace = True)

    df_arm_big.dropna(axis = 0,inplace = True)
    df_arm_small.dropna(axis = 0,inplace = True)
    df_arm_soso.dropna(axis = 0,inplace = True)

    df_chest_big.dropna(axis = 0,inplace = True)
    df_chest_small.dropna(axis = 0,inplace = True)
    df_chest_soso.dropna(axis = 0,inplace = True)

    df_shoulder_big.dropna(axis = 0,inplace = True)
    df_shoulder_small.dropna(axis = 0,inplace = True)
    df_shoulder_soso.dropna(axis = 0,inplace = True)
    
    # train df list
    hood_chongjang_train_lst = [df_chongjang_big, df_chongjang_small, df_chongjang_soso]
    hood_shoulder_train_lst = [df_shoulder_big, df_shoulder_small, df_shoulder_soso]
    hood_chest_train_lst = [df_chest_big, df_chest_small, df_chest_soso]
    hood_arm_train_lst = [df_arm_big, df_arm_small, df_arm_soso]
    
    return hood_chongjang_train_lst, hood_shoulder_train_lst, hood_chest_train_lst, hood_arm_train_lst

In [34]:
hood_chongjang_train_lst, hood_shoulder_train_lst, hood_chest_train_lst, hood_arm_train_lst = preprocessing(df)

* lr

In [28]:
import joblib
from sklearn.linear_model import LinearRegression
def lr_trainModel(lst,sizetype):
    user_pred_list = []
    for i in range(len(lst)):
        
        X = lst[i].iloc[:, :-1]
        y = lst[i].iloc[:, -1]

        xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size = 0.2, random_state = 42)
        #print(xTrain.shape, xTest.shape, yTrain.shape, yTest.shape)
        
        lr = LinearRegression()
        lr.fit(xTrain,yTrain)
        prediction = lr.predict(xTest)
        # mseOos = []
        # nTreeList = range(50, 500, 10)

        # for iTrees in tqdm(nTreeList, desc='iterate list'):
        #     depth = None
        #     RFModel = RandomForestRegressor(n_estimators=iTrees,
        #                                     max_depth=depth,
        #                                     oob_score=False, 
        #                                     random_state=531)
        #     RFModel.fit(xTrain, yTrain)
            
        #     #데이터 세트에 대한 MSE 누적
        #     prediction = RFModel.predict(xTest)
        #     mseOos.append(mean_squared_error(yTest, prediction))
            
        # regr = RandomForestRegressor(n_estimators = nTreeList[np.argmin(mseOos)],
        #                         random_state=531)
        # regr.fit(xTrain, yTrain)
        
        #prediction = regr.predict(xTest)
        if i % 3 == 1:
            print("MSE of small {}'s Model : {}".format(lst[i].columns[-1], mean_squared_error(yTest, prediction)))
            joblib.dump(lr, 'model/'+sizetype+'_small_lrModel.pkl')
        elif i % 3 == 2:
            print("MSE of soso {}'s Model : {}".format(lst[i].columns[-1], mean_squared_error(yTest, prediction)))
            joblib.dump(lr, 'model/'+sizetype+'_soso_lrModel.pkl')
        elif i % 3 == 0:
            print("MSE of big {}'s Model : {}".format(lst[i].columns[-1], mean_squared_error(yTest, prediction)))
            joblib.dump(lr, 'model/'+sizetype+'_big_lrModel.pkl')
            
        userTest = [[178, 76,0]]
        prediction_user = lr.predict(userTest)
        user_pred_list.append(prediction_user)
        print("User Test of {}'s Model : {}".format(lst[i].columns[-1], prediction_user))
    

In [29]:
sizetype = 'chongjang'
lr_trainModel(hood_chongjang_train_lst,sizetype)
sizetype = 'shoulder'
lr_trainModel(hood_shoulder_train_lst,sizetype)
sizetype = 'chest'
lr_trainModel(hood_chest_train_lst,sizetype)
sizetype = 'arm'
lr_trainModel(hood_arm_train_lst,sizetype)

MSE of big 총장's Model : 10.56246451429201
User Test of 총장's Model : [72.47986138]
MSE of small 총장's Model : 34.64142186013472
User Test of 총장's Model : [66.63667296]
MSE of soso 총장's Model : 29.829644338174713
User Test of 총장's Model : [71.16712188]
MSE of big 어깨너비's Model : 25.808701244945308
User Test of 어깨너비's Model : [64.08138523]
MSE of small 어깨너비's Model : 21.886753090743852
User Test of 어깨너비's Model : [62.61880571]
MSE of soso 어깨너비's Model : 27.78459477561879
User Test of 어깨너비's Model : [62.23460451]
MSE of big 가슴단면's Model : 17.305104278729896
User Test of 가슴단면's Model : [66.95859724]
MSE of small 가슴단면's Model : 27.838622143892806
User Test of 가슴단면's Model : [62.85977089]
MSE of soso 가슴단면's Model : 17.769704632618247
User Test of 가슴단면's Model : [64.48208286]
MSE of big 소매길이's Model : 39.14226190003229
User Test of 소매길이's Model : [62.03967739]
MSE of small 소매길이's Model : 36.32076057062744
User Test of 소매길이's Model : [60.46368233]
MSE of soso 소매길이's Model : 35.86978391163876
User

* gbr

In [30]:
import joblib
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import accuracy_score


def gbr_trainModel(lst,sizetype):
    user_pred_list = []
    for i in tqdm(range(len(lst))):
        
        X = lst[i].iloc[:, :-1]
        y = lst[i].iloc[:, -1]

        xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size = 0.2, random_state = 42)
        #print(xTrain.shape, xTest.shape, yTrain.shape, yTest.shape)
        
        
        gbr = GradientBoostingRegressor()

        parameters = {'learning_rate': [0.01,0.05,0.1,0.15],
                  'subsample'    : [0.9, 0.5, 0.2, 0.1],
                  'n_estimators' : [100,200,300,500],
                  'max_depth'    : [4,6,8,10]
                 }

        grid_gbr = GridSearchCV(estimator=gbr, param_grid = parameters, cv = 2, n_jobs=-1)
        grid_gbr.fit(xTrain,yTrain)
        print("The best estimator across ALL searched params:",grid_gbr.best_estimator_)
        
        model = grid_gbr.best_estimator_
        model.fit(xTrain,yTrain)
        prediction = model.predict(xTest)
        print('accuracy:{0:.4f}'.format(model.score(xTest,yTest)))
        
        if i % 3 == 1:
            print("MSE of small {}'s Model : {}".format(lst[i].columns[-1], mean_squared_error(yTest, prediction)))
            joblib.dump(model, 'model/'+sizetype+'_small_gbrModel.pkl')
        elif i % 3 == 2:
            print("MSE of soso {}'s Model : {}".format(lst[i].columns[-1], mean_squared_error(yTest, prediction)))
            joblib.dump(model, 'model/'+sizetype+'_soso_gbrModel.pkl')
        elif i % 3 == 0:
            print("MSE of big {}'s Model : {}".format(lst[i].columns[-1], mean_squared_error(yTest, prediction)))
            joblib.dump(model, 'model/'+sizetype+'_big_gbrModel.pkl')
            
        userTest = [[178, 76,0]]
        prediction_user = model.predict(userTest)
        user_pred_list.append(prediction_user)
        print("User Test of {}'s Model : {}".format(lst[i].columns[-1], prediction_user))
    

In [31]:
sizetype = 'chongjang'
gbr_trainModel(hood_chongjang_train_lst,sizetype)
sizetype = 'shoulder'
gbr_trainModel(hood_shoulder_train_lst,sizetype)
sizetype = 'chest'
gbr_trainModel(hood_chest_train_lst,sizetype)
sizetype = 'arm'
gbr_trainModel(hood_arm_train_lst,sizetype)

  0%|          | 0/3 [00:00<?, ?it/s]

The best estimator across ALL searched params: GradientBoostingRegressor(learning_rate=0.01, max_depth=4, n_estimators=200,
                          subsample=0.1)


 33%|███▎      | 1/3 [01:04<02:08, 64.14s/it]

accuracy:0.2051
MSE of big 총장's Model : 11.037546229977856
User Test of 총장's Model : [71.92009129]


 67%|██████▋   | 2/3 [01:26<00:39, 39.46s/it]

...

accuracy:0.0491
MSE of big 소매길이's Model : 38.8626674649251
User Test of 소매길이's Model : [61.67052208]


 67%|██████▋   | 2/3 [01:28<00:40, 40.69s/it]

The best estimator across ALL searched params: GradientBoostingRegressor(learning_rate=0.01, max_depth=4, subsample=0.2)
accuracy:0.0309
MSE of small 소매길이's Model : 39.62548496330436
User Test of 소매길이's Model : [60.46684276]
The best estimator across ALL searched params: GradientBoostingRegressor(learning_rate=0.01, max_depth=4, n_estimators=200,
                          subsample=0.5)


100%|██████████| 3/3 [03:41<00:00, 73.90s/it]

accuracy:0.0748
MSE of soso 소매길이's Model : 35.258900868943684
User Test of 소매길이's Model : [61.72855214]


# 가중치 구하는 법!

In [85]:
#df_chongjang_big #크다!!
#hood_chongjang_train_lst[1]

def decide_weight(lst,size_type):
    print('-------'+size_type+'-------')
    for i in range(len(lst)):

        X = hood_chongjang_train_lst[i].iloc[:, :-1]
        y = hood_chongjang_train_lst[i].iloc[:, -1]
        xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size = 0.2, random_state = 42)

        # a = range(0.1, 1.0, 0.1)
        # b = range(0.1, 1.0, 0.1)
        # c = 1 - a -b

        load_model_big = joblib.load('model/'+size_type+'_big_gbrModel.pkl')
        prediction_big = load_model_big.predict(xTest)#a
                
        load_model_small = joblib.load('model/'+size_type+'_small_gbrModel.pkl')
        prediction_small = load_model_small.predict(xTest)#b
                
        load_model_soso = joblib.load('model/'+size_type+'_soso_gbrModel.pkl')
        prediction_soso = load_model_soso.predict(xTest)#c

        mse_lst = []
        abc = []

        for a in np.arange(0.1, 1.0, 0.1):
            for b in np.arange(0.1, 1 - a, 0.1):
                
                c = 1 - (a + b)
                final_prediction = prediction_big * a + prediction_small* b + prediction_soso * c
                abc.append([a,b,c])
                mse_lst.append(mean_squared_error(yTest, final_prediction))
                
        if i % 3 == 1:
            print("small일 때! 최소 MSE:{},가중치(big,small,soso):{}".format(mse_lst[np.argmin(mse_lst)],abc[np.argmin(mse_lst)]))
            
        elif i % 3 == 2:
            print("soso일 때! 최소 MSE:{},가중치(big,small,soso):{}".format(mse_lst[np.argmin(mse_lst)],abc[np.argmin(mse_lst)]))
            
        elif i % 3 == 0:
            print("big일 때! 최소 MSE:{},가중치(big,small,soso):{}".format(mse_lst[np.argmin(mse_lst)],abc[np.argmin(mse_lst)]))
            
            
        abc[np.argmin(mse_lst)]
    print(len(mse_lst),len(abc)) 
#c[np.argmin(mseOos)]
        
        

In [86]:
size_type = 'chongjang'
decide_weight(hood_chongjang_train_lst,size_type)
size_type = 'chest'
decide_weight(hood_chest_train_lst,size_type)
size_type = 'shoulder'
decide_weight(hood_shoulder_train_lst,size_type)
size_type = 'arm'
decide_weight(hood_arm_train_lst,size_type)

-------chongjang-------
big일 때! 최소 MSE:10.806990951662156,가중치(big,small,soso):[0.5, 0.1, 0.4]
small일 때! 최소 MSE:35.28560618474201,가중치(big,small,soso):[0.1, 0.8, 0.09999999999999998]
soso일 때! 최소 MSE:13.726606344031598,가중치(big,small,soso):[0.2, 0.1, 0.7]
37 37
-------chest-------
big일 때! 최소 MSE:48.996038143783075,가중치(big,small,soso):[0.8, 0.1, 0.09999999999999998]
small일 때! 최소 MSE:47.457402586184266,가중치(big,small,soso):[0.8, 0.1, 0.09999999999999998]
soso일 때! 최소 MSE:44.76712161823022,가중치(big,small,soso):[0.8, 0.1, 0.09999999999999998]
37 37
-------shoulder-------
big일 때! 최소 MSE:86.9162408752424,가중치(big,small,soso):[0.8, 0.1, 0.09999999999999998]
small일 때! 최소 MSE:67.3979074200572,가중치(big,small,soso):[0.8, 0.1, 0.09999999999999998]
soso일 때! 최소 MSE:79.76257782222808,가중치(big,small,soso):[0.8, 0.1, 0.09999999999999998]
37 37
-------arm-------
big일 때! 최소 MSE:94.04744006997855,가중치(big,small,soso):[0.8, 0.1, 0.09999999999999998]
small일 때! 최소 MSE:72.91223825308847,가중치(big,small,soso):[0.1, 0.1, 0.

In [66]:
lst = []
for a in np.arange(0.0, 1.0, 0.1):
    for b in np.arange(0.0, 1.1 - a, 0.1):
            c = 1 - (a + b)
            lst.append([a,b,c])
            print(a,b,c)
print(len(lst))

0.0 0.0 1.0
0.0 0.1 0.9
0.0 0.2 0.8
0.0 0.30000000000000004 0.7
0.0 0.4 0.6
0.0 0.5 0.5
0.0 0.6000000000000001 0.3999999999999999
0.0 0.7000000000000001 0.29999999999999993
0.0 0.8 0.19999999999999996
0.0 0.9 0.09999999999999998
0.0 1.0 0.0
0.1 0.0 0.9
0.1 0.1 0.8
0.1 0.2 0.7
0.1 0.30000000000000004 0.6
0.1 0.4 0.5
0.1 0.5 0.4
0.1 0.6000000000000001 0.29999999999999993
0.1 0.7000000000000001 0.19999999999999996
0.1 0.8 0.09999999999999998
0.1 0.9 0.0
0.2 0.0 0.8
0.2 0.1 0.7
0.2 0.2 0.6
0.2 0.30000000000000004 0.5
0.2 0.4 0.3999999999999999
0.2 0.5 0.30000000000000004
0.2 0.6000000000000001 0.19999999999999996
0.2 0.7000000000000001 0.09999999999999987
0.2 0.8 0.0
0.30000000000000004 0.0 0.7
0.30000000000000004 0.1 0.6
0.30000000000000004 0.2 0.5
0.30000000000000004 0.30000000000000004 0.3999999999999999
0.30000000000000004 0.4 0.29999999999999993
0.30000000000000004 0.5 0.19999999999999996
0.30000000000000004 0.6000000000000001 0.09999999999999987
0.30000000000000004 0.7000000000000001